In [1]:
%cd ..

E:\システムトレード入門\trade_system_git_workspace


In [7]:
from pathlib import Path
import pandas as pd
from pathlib import Path
from pytz import timezone
import datetime
import numpy as np
from scipy.special import softmax

In [17]:
import bokeh.plotting
from bokeh.models import Range1d, LinearAxis, Div, HoverTool
from bokeh.io import show
from bokeh.io import output_notebook, reset_output, output_file
from bokeh.palettes import d3
output_notebook()

Loading BokehJS ...

In [6]:
from get_stock_price import StockDatabase

In [20]:
from utils import get_naive_datetime_from_datetime

In [11]:
from portfolio.trade_transformer import PortfolioTransformer, PortfolioRestrictorIdentity, FeeCalculatorPerNumber
from portfolio.price_supply import StockDBPriceSupplier

## データベース 

In [8]:
db_path = Path("E:/システムトレード入門/trade_system_git_workspace/db/sub_stock_db") / Path("sub_stock.db")
stock_db = StockDatabase(db_path)

## ポートフォリオの遷移 

In [27]:
jst_timezone = timezone("Asia/Tokyo")
start_datetime = jst_timezone.localize(datetime.datetime(2020,11,10,9,0,0))
stock_list = ["4755","9984","6701","7203","7267", "6502"]
episode_length = 500

price_supplier = StockDBPriceSupplier(stock_db=stock_db,
                                     ticker_names=stock_list,
                                     episode_length=episode_length,
                                     freq_str="5T",
                                     interpolate=True
                                    )

transformer = PortfolioTransformer(price_supplier=price_supplier,
                                   portfolio_restrictor=PortfolioRestrictorIdentity(),
                                   use_ohlc="Close",
                                   initial_portfolio_vector=None,
                                   initial_all_assets=1e6,
                                   fee_calculator=FeeCalculatorPerNumber(0)
                                  )


portfolio_state_list = []
initial_state, _ = transformer.reset(start_datetime, window=[-1,0,1])
portfolio_state_list.append(initial_state.partial("names", "now_price_array", "mean_cost_price_array", "portfolio_vector", "all_assets", "datetime"))

while True:
    action = softmax(np.abs(np.random.randn(1+len(stock_list))))
    portfolio_state, done = transformer.step(action)
    portfolio_state_list.append(portfolio_state.partial("names", "now_price_array", "mean_cost_price_array", "portfolio_vector", "all_assets", "datetime"))
    if done:
        break

In [28]:
portfolio_state_list

[PortfolioState(names=['yen', '4755', '9984', '6701', '7203', '7267', '6502'], key_currency_index=None, window=None, datetime=datetime.datetime(2020, 11, 10, 9, 0, tzinfo=<DstTzInfo 'Asia/Tokyo' JST+9:00:00 STD>), price_array=None, volume_array=None, now_price_array=array([1.0000e+00, 1.1280e+03, 7.0720e+03, 5.6800e+03, 7.3420e+03,
        2.9595e+03, 2.7800e+03]), portfolio_vector=array([1., 0., 0., 0., 0., 0., 0.]), mean_cost_price_array=array([1.0000e+00, 1.1280e+03, 7.0720e+03, 5.6800e+03, 7.3420e+03,
        2.9595e+03, 2.7800e+03]), all_assets=1000000.0),
 PortfolioState(names=['yen', '4755', '9984', '6701', '7203', '7267', '6502'], key_currency_index=None, window=None, datetime=datetime.datetime(2020, 11, 10, 9, 5, tzinfo=<DstTzInfo 'Asia/Tokyo' JST+9:00:00 STD>), price_array=None, volume_array=None, now_price_array=array([1.0000e+00, 1.1070e+03, 7.0190e+03, 5.7200e+03, 7.3360e+03,
        2.9415e+03, 2.8040e+03]), portfolio_vector=array([0.10671244, 0.1215606 , 0.15939091, 0.07

limit_output extension: Maximum message size of 100000 exceeded with 332820 characters

## ポートフォリオ遷移の可視化 

In [29]:
def make_y_limit(y_array, upper_ratio=0.1, lowwer_ratio=0.1):
    min_value = np.amin(y_array)
    max_value = np.amax(y_array)
    diff = max_value - min_value
    return min_value-lowwer_ratio*diff, max_value+upper_ratio*diff

In [30]:
def make_y_limit_multi(y_arrays, upper_ratio=0.1, lowwer_ratio=0.1):
    min_values = []
    max_values = []
    for y_array in y_arrays:
        min_values.append(np.amin(y_array))
        max_values.append(np.amax(y_array))
        
    min_value = min(min_values)
    max_value = max(max_values)
    diff = max_value - min_value
    
    return min_value-lowwer_ratio*diff, max_value+upper_ratio*diff

In [31]:
def make_ticker_text(ticker_value_array, ticker_names):
    div_text = ""
    text_sum_line = 150
    text_sum_count = 0

    for i, ticker_name in enumerate(ticker_names):
        div_text += ticker_name + "="
        text_sum_count += len(ticker_name)
        ticke_value_str = str(ticker_value_array[i])
        div_text += ticke_value_str
        text_sum_count += len(ticke_value_str)

        div_text += ", "
        text_sum_count += 2

        if text_sum_count > text_sum_line:
            div_text += "\n"
            text_sum_count = 0
            
    return div_text

In [32]:
def visualize_portfolio_transform_bokeh(portfolio_state_list, save_path=None, is_save=False, is_show=True, is_jupyter=True):
    # テータの取り出し
    ticker_names = portfolio_state_list[0].names
    colors = d3["Category20"][len(ticker_names)]

    all_price_array = np.stack([one_state.now_price_array for one_state in portfolio_state_list], axis=1)
    all_portfolio_vector = np.stack([one_state.portfolio_vector for one_state in portfolio_state_list], axis=1)
    all_mean_cost_price_array = np.stack([one_state.mean_cost_price_array for one_state in portfolio_state_list], axis=1)
    all_assets_array = np.array([one_state.all_assets for one_state in portfolio_state_list])
    all_datetime_array = np.array([get_naive_datetime_from_datetime(one_state.datetime) for one_state in portfolio_state_list])
    x = np.arange(0, len(portfolio_state_list))


    # sorceの作成
    portfolio_vector_source = {"x":x, "datetime":all_datetime_array}
    price_source_x = []
    price_source_y = []

    mean_cost_price_source_x = []
    mean_cost_price_source_y = []

    for i, ticker_name in enumerate(ticker_names):
        portfolio_vector_source[ticker_name] = all_portfolio_vector[i,:]

        price_source_x.append(x)
        price_source_y.append(all_price_array[i,:]/all_price_array[i,0])

        mean_cost_price_source_x.append(x)
        mean_cost_price_source_y.append(all_mean_cost_price_array[i,:]/all_mean_cost_price_array[i,0])

    # ホバーツールの設定
    #tool_tips = [("x", "@x")]
    tool_tips = [("datetime", "@datetime{%F %H:%M:%S}")]
    tool_tips.extend([(ticker_name, "@"+ticker_name+"{0.000}") for ticker_name in ticker_names])

    hover_tool = HoverTool(
        tooltips=tool_tips,
        formatters={'@datetime' : 'datetime'}
    )

    # 描画

    p1_text = Div(text=make_ticker_text(all_price_array[:,0], ticker_names))

    p1 = bokeh.plotting.figure(plot_width=1200,plot_height=500,title="正規化価格・ポートフォリオ")
    p1.add_tools(hover_tool)

    p1.extra_y_ranges = {"portfolio_vector": Range1d(start=0, end=3)}
    p1.add_layout(LinearAxis(y_range_name="portfolio_vector"), 'right')
    p1.vbar_stack(ticker_names, x='x', width=1, color=colors,y_range_name="portfolio_vector", source=portfolio_vector_source, legend_label=ticker_names, alpha=0.8)

    p1.multi_line(xs=price_source_x, ys=price_source_y, line_color=colors, line_width=2)
    y_min, y_max = make_y_limit_multi(price_source_y, lowwer_ratio=0.1, upper_ratio=0.1)
    y_min -= (y_max - y_min) * 0.66  #  ポートフォリオ割合のためのオフセット
    p1.y_range = Range1d(start=y_min, end=y_max)

    p1.yaxis[0].axis_label = "正規化価格"
    p1.yaxis[1].axis_label = "保有割合"

    p1.xaxis.major_label_overrides = {str(one_x) : str(all_datetime_array[i]) for i, one_x in enumerate(x)}

    p2_text = Div(text=make_ticker_text(all_mean_cost_price_array[:,0], ticker_names))

    p2 = bokeh.plotting.figure(plot_width=1200,plot_height=300,title="正規化平均取得価格・全資産")
    p2.multi_line(xs=mean_cost_price_source_x, ys=mean_cost_price_source_y, line_color=colors, line_width=2)
    y_min, y_max = make_y_limit_multi(mean_cost_price_source_y, lowwer_ratio=0.1, upper_ratio=0.1)
    p2.y_range = Range1d(start=y_min, end=y_max)

    y_max, y_min = make_y_limit(all_assets_array, upper_ratio=0.1, lowwer_ratio=0.1)
    p2.extra_y_ranges = {"all_assets": Range1d(start=y_max, end=y_min)}
    p2.add_layout(LinearAxis(y_range_name="all_assets"), 'right')
    p2.line(x, all_assets_array, color="red", legend_label="all_assets", line_width=4, y_range_name="all_assets")

    # 疑似的なレジェンドをつける
    for ticker_name, color in zip(ticker_names, colors):
        p2.line([], [], legend_label=ticker_name, color=color, line_width=2)

    p2.yaxis[0].axis_label = "正規化平均取得価格"
    p2.yaxis[1].axis_label = "全資産 [円]"

    p2.xaxis.major_label_overrides = {str(one_x) : str(all_datetime_array[i]) for i, one_x in enumerate(x)}

    created_figure = bokeh.layouts.column(p1_text, p1, p2_text, p2)

    if is_save:
            if save_path.suffix == ".png":
                bokeh.io.export_png(created_figure, filename=save_path)
            elif save_path.suffix == ".html":
                output_file(save_path)
                bokeh.io.save(created_figure, filename=save_path, title="trading process")    
            else:
                raise Exception("The suffix of save_path is must be '.png' or '.html'.")
    if is_show:
        try:
            reset_output()
            if is_jupyter:
                output_notebook()
            show(created_figure)
        except:
            if is_jupyter:
                output_notebook()
            show(created_figure)
        
    if not is_save and not is_show:
        raise Exception("is_save and is_show is False. This function do nothing")

In [33]:
visualize_portfolio_transform_bokeh(portfolio_state_list, save_path=Path("visualization/trade_transform.png"), is_save=False, is_show=True)

Loading BokehJS ...